# This file is for messing around

You've read enough, just get started or else you'll never really get a feel for it. 

Remaining items to figure out:

**1. How to implement material dispersion:**

For a wavelength-dependent, purely-real permittivity (i.e., with no loss) which can be represented via the Sellmeier equation:

$$
\epsilon(\lambda) = 1 + \sum_{n} \frac{B_n \lambda^2}{\lambda^2 - C_n}
$$

where $\lambda$ is the vacuum wavelength, each term containing two coefficients ($B_n$ and $C_n$) can be directly transferred to a Lorentzian polarization field using a simple substitution of variables: $\omega_n = 1 \big/ \sqrt{C_n}$, $\gamma_n = 0$ and $\sigma_n = B_n$. Several examples of importing Sellmeier coefficients from published fitting data including [germanium](https://github.com/NanoComp/meep/blob/master/python/materials.py#L884-L901) (Ge) and [gallium nitride](https://github.com/NanoComp/meep/blob/master/python/materials.py#L1162-L1188) (GaN) are provided in the [Materials Library](https://meep.readthedocs.io/en/latest/Materials/#materials-library).

In [2]:
import meep as mp
import matplotlib.pyplot as pltd
import numpy as np